# Imports

In [1]:
!pip install sklearn
!pip install nltk

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, cross_val_score, GridSearchCV
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

In [3]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Import Data

In [4]:
df_train = pd.read_csv('ferramenta_competition_train.csv')


In [5]:
df_test = pd.read_csv('ferramenta_competition_test_without_labels.csv')

In [6]:
df_train.head(1)

,image,title,description,class,f0,f1,f2,f3,f4,f5,...,f246,f247,f248,f249,f250,f251,f252,f253,f254,f255
0,181587231.jpg,record irwin irwin record 96 morsa da banco pe...,irwin record 96 morsa da banco per tubi 3 152 mm,_@morsetto_strettoio_,0.0,0.0,0.0,0.254326,0.0,0.2474,...,0.237102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.063287,0.0


In [7]:
df_test.head(1)

,image,title,description,f0,f1,f2,f3,f4,f5,f6,...,f246,f247,f248,f249,f250,f251,f252,f253,f254,f255
0,193298868.jpg,durlach magnat plus morsa da banco con mds 160...,durlach magnat plus morsa da banco con mds 160...,0.084366,0.132483,0.0,0.616159,0.0,0.358702,0.033038,...,0.249871,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.258719,0.0


In [8]:
y = df_train['class']

In [9]:
X = df_train.drop(columns = ['class', 'image'])

In [10]:
X.head(1)

,title,description,f0,f1,f2,f3,f4,f5,f6,f7,...,f246,f247,f248,f249,f250,f251,f252,f253,f254,f255
0,record irwin irwin record 96 morsa da banco pe...,irwin record 96 morsa da banco per tubi 3 152 mm,0.0,0.0,0.0,0.254326,0.0,0.2474,0.0,0.039258,...,0.237102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.063287,0.0


# Preprocessing

In [11]:
from re import search
# Código extrído de la Ayudantía 3
def my_pre_processer(text, debug = False):
    stopwords_it = stopwords.words('italian')
    results = []
    if (debug):
        print("Tokenizer result: ", tokenizer(text))
    for token in tokenizer(text):
        clean_token = token.lower().strip('-').strip('_')
        if remove_stops_here and (clean_token in stopwords_it):
          continue
        #token_pro = stemmer.stem(clean_token) #podemos probar stemming en vez de lematizacion
        token_pro = lemmatizer.lemmatize(clean_token) 
        if len(token_pro) > 2 and not token_pro[0].isdigit() and not bool(search(r'\d', token_pro)): #elimina palabra largo menor a 2
            results.append(token_pro)
    return results

In [12]:
remove_stops_here = False
tokenizer = TfidfVectorizer().build_tokenizer()
lemmatizer = WordNetLemmatizer()

In [13]:
ngram_max = 4
max_features = 10000 # 10mil numeros reales
max_df=0.8
min_df=1

stopwords_it = stopwords.words('italian')+['avra', 'avro', 'fara', 'faro', 'perche', 'piu', 'quantum', 'sara', 'saro', 'stara', 'staro']

vectorizer = TfidfVectorizer(stop_words = stopwords_it, tokenizer=my_pre_processer, min_df=min_df, max_df=max_df, max_features=max_features,
                             binary=False, use_idf=True, smooth_idf=True, norm=None,#, ngram_range=(1, ngram_max),
                             strip_accents = 'unicode', lowercase = True, 
                            )

vectorizer_title = TfidfVectorizer(stop_words = stopwords_it, tokenizer=my_pre_processer, min_df=min_df, max_df=max_df, max_features=max_features,
                             binary=False, use_idf=True, smooth_idf=True, norm=None,#, ngram_range=(1, ngram_max),
                             strip_accents = 'unicode', lowercase = True, 
                            )

In [14]:
# Creación del vocabulario de las categorías
vocabulary = X['description'].append(df_test['description'])
vocabulary.describe()

count                                                  4207
unique                                                 3788
top       vernice spray brillante a rapida essicazione p...
freq                                                     48
Name: description, dtype: object

In [15]:
# Creación del vocabulario de los titulos
vocabulary_title = X['title'].append(df_test['title'])

In [16]:
vectorizer.fit(vocabulary)

/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(max_df=0.8, max_features=10000, norm=None,
                stop_words=['ad', 'al', 'allo', 'ai', 'agli', 'all', 'agl',
                            'alla', 'alle', 'con', 'col', 'coi', 'da', 'dal',
                            'dallo', 'dai', 'dagli', 'dall', 'dagl', 'dalla',
                            'dalle', 'di', 'del', 'dello', 'dei', 'degli',
                            'dell', 'degl', 'della', 'delle', ...],
                strip_accents='unicode',
                tokenizer=<function my_pre_processer at 0x407fabc820>)

In [17]:
vectorizer_title.fit(vocabulary_title)

TfidfVectorizer(max_df=0.8, max_features=10000, norm=None,
                stop_words=['ad', 'al', 'allo', 'ai', 'agli', 'all', 'agl',
                            'alla', 'alle', 'con', 'col', 'coi', 'da', 'dal',
                            'dallo', 'dai', 'dagli', 'dall', 'dagl', 'dalla',
                            'dalle', 'di', 'del', 'dello', 'dei', 'degli',
                            'dell', 'degl', 'della', 'delle', ...],
                strip_accents='unicode',
                tokenizer=<function my_pre_processer at 0x407fabc820>)

In [18]:
# Convertir los textos de entrenamiento
X_vectorized = vectorizer.transform(X['description'])
X_vectorized.shape

(2524, 4218)

In [19]:
X_title_vectorized = vectorizer_title.transform(X['title'])
X_title_vectorized.shape

(2524, 2906)

In [ ]:
#Creación de dataframes
train_feature_names = vectorizer.get_feature_names()
dense = X_vectorized.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=[f'cat_{col}' for col in train_feature_names])

/opt/conda/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
train_feature_names = vectorizer_title.get_feature_names_out()
dense = X_title_vectorized.todense()
denselist = dense.tolist()
title = pd.DataFrame(denselist, columns=[f'title_{col}' for col in train_feature_names])

In [ ]:
X_tmp = X
image_data = X_tmp.drop(columns = ['title', 'description'])

In [ ]:
df = pd.concat([title, df, image_data], axis=1).reindex(df.index)
df.head(1)

# Model

In [ ]:
# Tunning de parametros

In [ ]:
model = MultinomialNB()

In [ ]:
cv = ShuffleSplit(n_splits=5, test_size=0.33, random_state=0)

In [ ]:
parameters = {'alpha': list(range(1,11))}

In [ ]:
clf = GridSearchCV(model, parameters, cv = cv)

In [ ]:
clf.fit(df, y)

In [ ]:
clf.best_estimator_.alpha

In [ ]:
parameters = {'alpha': np.linspace(2,3,5)}

In [ ]:
clf = GridSearchCV(model, parameters, cv = cv)

In [ ]:
clf.fit(df, y)

In [ ]:
clf.best_estimator_.alpha

In [ ]:
# Cross Validation usando el mejor parametro

In [ ]:
model = MultinomialNB(alpha=2.25)

In [ ]:
scores = cross_val_score(model, df, y, cv=cv)
scores

In [ ]:
scores.mean()

## Testing

In [ ]:
# Entrenamiento y Testing usando división de los datos de entrenamiento

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.33, random_state=0)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

## Run classification

In [ ]:
# Se reproducen los mismos pasos anteriores de preprocesamiento y 
# se hace la predicción de las categorias usando la data de prueba

In [ ]:
X_test_vect = vectorizer.transform(df_test['description'])
X_test_vect.shape

In [ ]:
X_title_vectorized = vectorizer_title.transform(df_test['title'])
X_title_vectorized.shape

In [ ]:
test_feature_names = vectorizer.get_feature_names_out()
dense = X_test_vect.todense()
denselist = dense.tolist()
x_desc = pd.DataFrame(denselist, columns=[f'cat_{col}' for col in test_feature_names])
x_desc

In [ ]:
test_feature_names = vectorizer_title.get_feature_names_out()
dense = X_title_vectorized.todense()
denselist = dense.tolist()
x_title = pd.DataFrame(denselist, columns=[f'title_{col}' for col in test_feature_names])
x_title

In [ ]:
X_tmp = df_test
image_data = X_tmp.drop(columns = ['title', 'description', 'image'])
image_data.head(1)

In [ ]:
x_data = pd.concat([x_title, x_desc, image_data], axis=1).reindex(x_desc.index)
x_data.head(1)

In [ ]:
y_pred = model.predict(x_data)
y_pred

In [ ]:
images = df_test['image']
images

In [ ]:
import csv

In [ ]:
with open('submisson.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(['image', 'class'])
    
    for i in range(images.shape[0]):
        writer.writerow([images[i], y_pred[i]])